In [1]:
import pandas as pd
#import matplotlib.pyplot as plt
#import seaborn as sns
#import warnings
#warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
#import gensim
#from gensim.models.doc2vec import Doc2Vec, LabeledSentence 
import numpy as np
import copy
#%matplotlib inline

In [2]:
def load_data():
    train_ori = pd.read_table("round1_ijcai_18_train_20180301.txt",delim_whitespace=True)
    test_ori = pd.read_table('round1_ijcai_18_test_a_20180301.txt',delim_whitespace = True)
    return train_ori, test_ori

In [3]:
train_ori, test_ori = load_data()
train_ori = train_ori.drop_duplicates()#训练集去重
train_1 = train_ori.copy(deep = True)
test_1 = test_ori.copy(deep = True)

In [4]:
print(test_ori.shape)
print(test_1.shape)
print(train_ori.shape)
print(train_1.shape)

(18371, 26)
(18371, 26)
(478111, 27)
(478111, 27)


In [5]:
train_y = train_1["is_trade"]
train_1 = train_1.drop("is_trade",axis = 1)

In [27]:
data = pd.concat([train_1,test_1])
data = data.reset_index(drop = True)
print(data.shape)

(496482, 26)


In [28]:
data.isnull().sum()#无缺失值

instance_id                  0
item_id                      0
item_category_list           0
item_property_list           0
item_brand_id                0
item_city_id                 0
item_price_level             0
item_sales_level             0
item_collected_level         0
item_pv_level                0
user_id                      0
user_gender_id               0
user_age_level               0
user_occupation_id           0
user_star_level              0
context_id                   0
context_timestamp            0
context_page_id              0
predict_category_property    0
shop_id                      0
shop_review_num_level        0
shop_review_positive_rate    0
shop_star_level              0
shop_score_service           0
shop_score_delivery          0
shop_score_description       0
dtype: int64

# 处理item_category_list这个feature

In [29]:
data.head(3)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description
0,108641074714126964,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
1,5754713551599725161,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537243232,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
2,842679481291040981,3412720377098676069,7908382889764677758;5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537211052,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0


In [30]:
cate_list = list(data['item_category_list'])

In [31]:
#分号前的category1
cate_1_list = []
for i in range(len(cate_list)):
    cate_1_list.append(cate_list[i].split(";")[0])
cate_1_list[:3]

['7908382889764677758', '7908382889764677758', '7908382889764677758']

In [32]:
set(cate_1_list)#发现所有样本都一样 因此去掉该子字段的信息

{'7908382889764677758'}

In [33]:
#分号后的category2
cate_2_list = []
for i in range(len(cate_list)):
    cate_2_list.append(cate_list[i].split(";")[1])
cate_2_list[:3]

['5799347067982556520', '5799347067982556520', '5799347067982556520']

In [34]:
set(cate_2_list)#有区别 因此用该子字段的信息代替原来的feature

{'1968056100269760729',
 '2011981573061447208',
 '22731265849056483',
 '2436715285093487584',
 '2642175453151805566',
 '3203673979138763595',
 '4879721024980945592',
 '509660095530134768',
 '5755694407684602296',
 '5799347067982556520',
 '7258015885215914736',
 '8277336076276184272',
 '8710739180200009128'}

In [35]:
data["item_category_list"] = cate_2_list

In [36]:
data.head(3)

,instance_id,item_id,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,...,context_timestamp,context_page_id,predict_category_property,shop_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description
0,108641074714126964,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537236544,4006,5799347067982556520:-1;509660095530134768:-1;5...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
1,5754713551599725161,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537243232,4001,5799347067982556520:9172976955054793469;790838...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0
2,842679481291040981,3412720377098676069,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,...,1537211052,4001,5799347067982556520:5131280576272319091;725801...,6765930309048922341,4,1.0,5002,1.0,1.0,1.0


# item_property_list这个feature与pred_property这个集合取交集 再除以真实property_list集合的势

In [37]:
data.columns

Index(['instance_id', 'item_id', 'item_category_list', 'item_property_list',
       'item_brand_id', 'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_id', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description'],
      dtype='object')

In [38]:
pred_cate_prop_list = list(data['predict_category_property'])
#用；分解每个cate：property
pcp_list = []
for i in pred_cate_prop_list:
    pcp_list.append(i.split(';'))
#再用：将cate，property分开
pcp_list_2 = []
for i in pcp_list:
    tmp = []
    for k in i:
        tmp.append(k.split(":"))
    pcp_list_2.append(tmp)
#每一行的category
each_category_list = []
for i in pcp_list_2:
    tmp = []
    for k in i:
        tmp.append(k[0])
    each_category_list.append(tmp)
#每一行的property
each_property_list = []
for i in pcp_list_2:
    tmp = []
    for k in i:
        tmp.extend(k[1:])
    each_property_list.append(tmp)
def property_clean(doc):
    doc_trans = []
    for i in range(len(doc)):
        temp = []
        for j in range(len(doc[i])):
            for k in range(len(doc[i][j].split(","))):
                temp.append(doc[i][j].split(",")[k])
        doc_trans.append(temp)
    return doc_trans
property_clean_1 = property_clean(each_property_list)
each_property_list_prep = [list(set(x)) for x in property_clean_1]
print(each_category_list[:3])
print("======================")
print(each_property_list_prep[:3])

[['5799347067982556520', '509660095530134768', '5755694407684602296', '8277336076276184272', '7908382889764677758'], ['5799347067982556520', '7908382889764677758'], ['5799347067982556520', '7258015885215914736', '7908382889764677758']]
[['-1', '9148482949976129397'], ['1787573075717641245', '5195139481388729954', '9148482949976129397', '9172976955054793469', '4621934203383159480'], ['5131280576272319091']]


In [39]:
lenth_cate_pred = []
for i in range(len(each_category_list)):
    lenth_cate_pred.append(len(each_category_list[i]))
lenth_prop_pred = []
for i in range(len(each_property_list_prep)):
    lenth_prop_pred.append(len(each_property_list_prep[i]))

In [40]:
data['lenth_cate_pred'] = lenth_cate_pred
data['lenth_prop_pred'] = lenth_prop_pred

In [41]:
union_property = []
for i in range(len(data)):
    union_property.append(len((set(data.loc[i,"item_property_list"].split(";"))) & \
     set(each_property_list_prep[i]))/len(set(data.loc[i,"item_property_list"].split(";"))))

In [44]:
lenth_prop = []
for i in range(len(data)):
    lenth_prop.append(len(set(data.loc[i,"item_property_list"].split(";"))))

In [45]:
data["union_property"] = union_property
data["lenth_prop"] = lenth_prop

In [46]:
union_category = []
for i in range(len(data)):
    union_category.append(len((set([data.loc[i,"item_category_list"]])) & \
     set(each_category_list[i])))

In [47]:
lenth_cate = []
for i in range(len(data)):
    lenth_cate.append(len(set(data.loc[i,"item_category_list"].split(";"))))

In [48]:
data["union_category"] = union_category
data["lenth_cate"] = lenth_cate

In [49]:
# #分号分割后的所有属性放到一起
# property_list_all = []
# for i in range(len(property_list)):
#     for j in range(len(property_list[i].split(";"))):
#         property_list_all.append(property_list[i].split(";")[j])
# #看看有多少种不同的属性
# property_distinct = list(set(property_list_all))
# print(len(property_distinct))

In [50]:
# #将原始的数据转换成doc2vec可用的lists of list的格式
# property_list_pre = []
# for i in range(len(property_list)):
#     temp = []
#     temp.append(property_list[i])
#     property_list_pre.append(temp)
# pre_process = [[word for word in document[0].split(';')] for document in property_list_pre]

In [51]:
# #看看拆分结果
# print(property_list_pre[:2])
# print("-"*50)
# print(pre_process[:2])

In [52]:
# def doc2vec(doc, size, window = 1, min_count = 1):
#     """
#     doc:lists of list格式
#     size:目标vector大小 
#     """
#     TaggededDocument = gensim.models.doc2vec.TaggedDocument  
#     labeled_doc = []  
#     for i, text in enumerate(doc):  
#         document = TaggededDocument(text, tags=[i])  
#         labeled_doc.append(document)
#     model_dm = Doc2Vec(labeled_doc, min_count = min_count, window = window, size = size, 
#                        negative=5, workers=4)
#     array_property = model_dm.infer_vector(pre_process[0])
#     for i in doc[1:]:
#         temp = model_dm.infer_vector(i)
#         array_property = np.vstack((array_property,temp))
#     dataframe_property = pd.DataFrame(array_property)
#     return dataframe_property

In [53]:
# dataframe_property = doc2vec(pre_process, 10)
# dataframe_property.head(3)

In [54]:
# dataframe_property.shape

In [55]:
data.shape

(496482, 32)

In [57]:
# dataframe_property.columns = ["prop1","prop2","prop3","prop4","prop5",
#                              "prop6","prop7","prop8","prop9","prop10",]

In [58]:
# data_1 = pd.concat([data, dataframe_property],axis=1)

In [59]:
len(set(list(data["item_brand_id"])))#商品品牌暂时不处理

2075

In [60]:
data_1 = data.drop(['instance_id','item_id','user_id'],axis = 1)
data_1.head(3)

,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_gender_id,user_age_level,...,shop_star_level,shop_score_service,shop_score_delivery,shop_score_description,lenth_cate_pred,lenth_prop_pred,union_property,lenth_prop,union_category,lenth_cate
0,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,1,1003,...,5002,1.0,1.0,1.0,5,2,0.000000,22,1,1
1,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,0,1002,...,5002,1.0,1.0,1.0,2,5,0.045455,22,1,1
2,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,0,1003,...,5002,1.0,1.0,1.0,3,1,0.045455,22,1,1


In [61]:
data_1.shape

(496482, 29)

# 处理predict_category_property这个feature

In [62]:
#pred_cate_prop_list = list(data_1['predict_category_property'])

In [63]:
#pred_cate_prop_list[:3]

In [64]:
# #用；分解每个cate：property
# pcp_list = []
# for i in pred_cate_prop_list:
#     pcp_list.append(i.split(';'))
# #再用：将cate，property分开
# pcp_list_2 = []
# for i in pcp_list:
#     tmp = []
#     for k in i:
#         tmp.append(k.split(":"))
#     pcp_list_2.append(tmp)

In [65]:
# pcp_list_2[:2]

In [66]:
# #每一行的category
# each_category_list = []
# for i in pcp_list_2:
#     tmp = []
#     for k in i:
#         tmp.append(k[0])
#     each_category_list.append(tmp)

In [67]:
# each_category_list[:3]

In [68]:
# dataframe_pred_cate = doc2vec(each_category_list, 5)
# dataframe_pred_cate.head(3)

In [69]:
# dataframe_pred_cate.columns = ["pred_cate1","pred_cate2","pred_cate3",
#                                "pred_cate4","pred_cate5"]

In [70]:
# data_2 = pd.concat([data_1, dataframe_pred_cate],axis=1)
# data_2.head(3)

In [71]:
# data_2.shape

In [72]:
# #每一行的property
# each_property_list = []
# for i in pcp_list_2:
#     tmp = []
#     for k in i:
#         tmp.extend(k[1:])
#     each_property_list.append(tmp)

In [73]:
# each_property_list[:2]

In [74]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans

In [75]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans
# property_clean_1 = property_clean(each_property_list)
# property_clean_1[:2]

In [76]:
# def property_clean(doc):
#     doc_trans = []
#     for i in range(len(doc)):
#         temp = []
#         for j in range(len(doc[i])):
#             for k in range(len(doc[i][j].split(","))):
#                 temp.append(doc[i][j].split(",")[k])
#         doc_trans.append(temp)
#     return doc_trans
# property_clean_1 = property_clean(each_property_list)
# property_clean_1[:2]
# each_property_list_prep = [list(set(x)) for x in property_clean_1]
# each_property_list_prep[:2]

In [77]:
# dataframe_pred_prop = doc2vec(each_property_list_prep, 10)
# dataframe_pred_prop.head(3)

In [78]:
# dataframe_pred_prop.columns = ["pred_prop1","pred_prop2","pred_prop3","pred_prop4","pred_prop5",
#                               "pred_prop6","pred_prop7","pred_prop8","pred_prop9","pred_prop10",]
# data_3 = pd.concat([data_2, dataframe_pred_prop],axis=1)
# data_3.head(3)

In [79]:
# data_3.shape

In [80]:
# data_3.to_csv("data_nlp.csv")

-----

# context_timestamp特征处理

In [81]:
# data_3 = pd.read_csv("data_nlp.csv")

In [82]:
def time_value(time_stamp):
    hour = int(time_stamp.strftime("%H:%M:%S")[:2]) * 3600
    minus = int(time_stamp.strftime("%H:%M:%S")[3:5]) * 60
    sec = int(time_stamp.strftime("%H:%M:%S")[6:8])
    return hour + minus + sec

In [83]:
import datetime 
time_value_list = []
day_list = []
for i in range(len(data_1['context_timestamp'])):
    time_stamp = datetime.datetime.utcfromtimestamp(data_1['context_timestamp'][i])
    day = time_stamp.strftime("%Y-%m-%d")[-2:]
    time_value_list.append(time_value(time_stamp))
    day_list.append(day)

In [84]:
data_1['context_timestamp'] = time_value_list
data_1["day"] = day_list
data_1.head(3)

,item_category_list,item_property_list,item_brand_id,item_city_id,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_gender_id,user_age_level,...,shop_score_service,shop_score_delivery,shop_score_description,lenth_cate_pred,lenth_prop_pred,union_property,lenth_prop,union_category,lenth_cate,day
0,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,1,1003,...,1.0,1.0,1.0,5,2,0.000000,22,1,1,18
1,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,0,1002,...,1.0,1.0,1.0,2,5,0.045455,22,1,1,18
2,5799347067982556520,2072967855524022579;5131280576272319091;263639...,1975590437749032870,3948283326616421003,3,3,4,14,0,1003,...,1.0,1.0,1.0,3,1,0.045455,22,1,1,17


In [85]:
data_2 = data_1.copy(deep = True)

In [86]:
#增加一列 timestamp bin
data_2['context_timestamp_bin'] = pd.cut(list(data_2['context_timestamp']),24)

In [87]:
data_2.columns

Index(['item_category_list', 'item_property_list', 'item_brand_id',
       'item_city_id', 'item_price_level', 'item_sales_level',
       'item_collected_level', 'item_pv_level', 'user_gender_id',
       'user_age_level', 'user_occupation_id', 'user_star_level', 'context_id',
       'context_timestamp', 'context_page_id', 'predict_category_property',
       'shop_id', 'shop_review_num_level', 'shop_review_positive_rate',
       'shop_star_level', 'shop_score_service', 'shop_score_delivery',
       'shop_score_description', 'lenth_cate_pred', 'lenth_prop_pred',
       'union_property', 'lenth_prop', 'union_category', 'lenth_cate', 'day',
       'context_timestamp_bin'],
      dtype='object')

In [88]:
data_2.shape

(496482, 31)

In [166]:
#进一步去掉无用的feature
data_3 = data_2.drop(['context_id','shop_id','predict_category_property',
                     'item_property_list','item_brand_id'],axis = 1)

In [167]:
# feature_dummy = ["context_timestamp_bin","user_occupation_id","user_age_level",
#                 "user_gender_id",'item_city_id','item_category_list',"day",
#                  "item_pv_level","user_star_level","context_page_id","shop_review_num_level",
#                 "shop_star_level","item_sales_level","item_collected_level","item_price_level"]

feature_dummy = ["context_timestamp_bin","user_occupation_id","user_age_level",
                "user_gender_id",'item_city_id','item_category_list',"day",
                ]

In [168]:
# data_3[feature_dummy] = data_3[feature_dummy].astype(str)

In [169]:
data_3["user_occupation_id"] = data_3["user_occupation_id"].astype(str)
data_3["user_age_level"] = data_3["user_age_level"].astype(str)
data_3["user_gender_id"] = data_3["user_gender_id"].astype(str)
data_3['item_city_id'] = data_3['item_city_id'].astype(str)
data_3['item_category_list'] = data_3['item_category_list'].astype(str)
data_3['day'] = data_3['day'].astype(str)
#data_3['item_brand_id'] = data_3['item_brand_id'].astype(str)

# data_3["item_pv_level"] = data_3["item_pv_level"].astype(str)
# data_3["user_star_level"] = data_3["user_star_level"].astype(str)
# data_3["context_page_id"] = data_3["context_page_id"].astype(str)
# data_3["shop_review_num_level"] = data_3["shop_review_num_level"].astype(str)
# data_3["shop_star_level"] = data_3["shop_star_level"].astype(str)
# data_3["item_sales_level"] = data_3["item_sales_level"].astype(str)
# data_3["item_collected_level"] = data_3["item_collected_level"].astype(str)
# data_3["item_price_level"] = data_3["item_price_level"].astype(str)

In [170]:
data_4 = pd.concat([data_3,pd.get_dummies(data_3[feature_dummy])],axis =1)
data_4 = data_4.drop(feature_dummy,axis = 1)

In [171]:
data_4.shape

(496482, 211)

In [172]:
# feature_std = ['item_price_level','shop_score_description','shop_score_delivery',
#               'shop_score_service','shop_star_level','shop_review_positive_rate',
#               'shop_review_num_level','context_page_id','user_star_level',
#                'item_pv_level','item_collected_level','item_sales_level',
#                'union_property','query_complexity_prope','union_category', 
#                'query_complexity_cate','lenth_cate_pred', 'lenth_prop_pred'
#               ]
feature_std = ['shop_score_description','shop_score_delivery','shop_score_service',
               'shop_review_positive_rate','union_property','lenth_prop',
               'union_category', 'lenth_cate','lenth_cate_pred','lenth_prop_pred',
               'context_timestamp',"item_pv_level","user_star_level","context_page_id",
               "shop_review_num_level","shop_star_level","item_sales_level",
               "item_collected_level","item_price_level"
               ]

In [173]:
from sklearn.preprocessing import StandardScaler
std = StandardScaler() 
std.fit(data_4[feature_std].values)
data_4[feature_std] = std.transform(list(data_4[feature_std].values))       
print(data_4.shape)

(496482, 211)


In [174]:
data_4.head(3)

,item_price_level,item_sales_level,item_collected_level,item_pv_level,user_star_level,context_timestamp,context_page_id,shop_review_num_level,shop_review_positive_rate,shop_star_level,...,item_category_list_8710739180200009128,day_17,day_18,day_19,day_20,day_21,day_22,day_23,day_24,day_25
0,-3.475377,-3.1379,-3.28112,-1.444348,0.035294,-1.415444,0.656024,-4.162958,0.435111,-4.310423,...,0,0,1,0,0,0,0,0,0,0
1,-3.475377,-3.1379,-3.28112,-1.444348,0.057759,-1.068065,-0.587808,-4.162958,0.435111,-4.310423,...,0,0,1,0,0,0,0,0,0,0
2,-3.475377,-3.1379,-3.28112,-1.444348,0.042782,1.748157,-0.587808,-4.162958,0.435111,-4.310423,...,0,1,0,0,0,0,0,0,0,0


------

In [175]:
train = data_4.loc[:478110,:]
test = data_4.loc[478111:,:]

In [176]:
train.shape

(478111, 211)

In [177]:
test.shape

(18371, 211)

In [178]:
train_y.shape

(478111,)

In [179]:
#Common Model Algorithms
from sklearn import svm, tree, linear_model, neighbors, naive_bayes, \
                    ensemble, discriminant_analysis, gaussian_process
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
#Common Model Helpers
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn import feature_selection
from sklearn import model_selection
from sklearn import metrics
#Visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from pandas.tools.plotting import scatter_matrix
%matplotlib inline
mpl.style.use('ggplot')
sns.set_style('white')
pylab.rcParams['figure.figsize'] = 12,8

In [182]:
MLA = [
#     ensemble.AdaBoostClassifier(),
#     ensemble.BaggingClassifier(),
#     ensemble.ExtraTreesClassifier(),
#     ensemble.GradientBoostingClassifier(),
#     ensemble.RandomForestClassifier(),
#     gaussian_process.GaussianProcessClassifier(),
#     linear_model.LogisticRegressionCV(),
#     linear_model.PassiveAggressiveClassifier(),
#     linear_model.RidgeClassifierCV(),
#     linear_model.SGDClassifier(),
#     linear_model.Perceptron(),
#     svm.SVC(probability=True),
#     svm.NuSVC(probability=True),
    LGBMClassifier(num_leaves=63, max_depth=7, n_estimators=80, n_jobs=20)
    #LGBMClassifier()
    ]

In [183]:
MLA

[LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
         learning_rate=0.1, max_depth=7, min_child_samples=20,
         min_child_weight=0.001, min_split_gain=0.0, n_estimators=80,
         n_jobs=20, num_leaves=63, objective=None, random_state=None,
         reg_alpha=0.0, reg_lambda=0.0, silent=True, subsample=1.0,
         subsample_for_bin=200000, subsample_freq=1)]

In [184]:
train = train.reset_index(drop = True)
test = test.reset_index(drop = True)

In [185]:
cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .7, 
                                        random_state = 42 ) 

MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train log_loss Mean', 
               'MLA Test log_loss Mean' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns)
row_index = 0
for alg in MLA:
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
    
    cv_results = model_selection.cross_validate(alg, train, 
                                                train_y,cv  = cv_split,
                                                scoring='neg_log_loss')

    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train log_loss Mean'] = -cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test log_loss Mean'] = -cv_results['test_score'].mean()   
    MLA_compare.loc[row_index, 'MLA Test log_loss 3*STD'] = -cv_results['test_score'].std()*3   #let's know the worst that can happen!
    
    alg.fit(train, train_y)
    MLA_predict = alg.predict(test)
    print(MLA_name,"complete...")
    
    row_index+=1

MLA_compare.sort_values(by = ['MLA Test log_loss Mean'], ascending = False, inplace = True)
MLA_compare

i:\anaconda\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


LGBMClassifier complete...


i:\anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


,MLA Name,MLA Parameters,MLA Train log_loss Mean,MLA Test log_loss Mean,MLA Time,MLA Test log_loss 3*STD
0,LGBMClassifier,"{'boosting_type': 'gbdt', 'class_weight': None...",0.0817816,0.0893604,14.9693,-0.002227


In [186]:
predict_prob = alg.predict_proba(test)[:,1]

In [187]:
submission = pd.read_table("round1_ijcai_18_result_demo_20180301.txt")

In [188]:
submission_final = pd.DataFrame(columns=submission.columns)

In [189]:
submission_final["instance_id"] = test_1["instance_id"]
submission_final["predicted_score"] = predict_prob
submission_final = submission_final.drop("instance_id predicted_score",axis = 1)
submission_final

,instance_id,predicted_score
0,2475218615076601065,0.007404
1,398316874173557226,0.021451
2,6586402638209028583,0.018573
3,1040996105851528465,0.022394
4,6316278569655873454,0.020663
5,868158305045921978,0.020491
6,5713520501786699854,0.013487
7,932945015407923184,0.008327
8,1919197847086752313,0.010393
9,304887065966615346,0.011244


In [190]:
from datetime import date

In [191]:
today = date.today().strftime('%Y%m%d')

In [192]:
submission_final.to_csv("submission_"+today+".csv",index = False,sep = ' ')

In [193]:
submission_final

,instance_id,predicted_score
0,2475218615076601065,0.007404
1,398316874173557226,0.021451
2,6586402638209028583,0.018573
3,1040996105851528465,0.022394
4,6316278569655873454,0.020663
5,868158305045921978,0.020491
6,5713520501786699854,0.013487
7,932945015407923184,0.008327
8,1919197847086752313,0.010393
9,304887065966615346,0.011244
